In [359]:
import os
import re
from datetime import datetime
current_datetime = datetime.now().strftime('%Y%m%d.%H%M')

import numpy as np
import pandas as pd

# for the notebook rendering 
from IPython.display import display, HTML, FileLink, Markdown
from IPython.display import FileLink as FL
from IPython.display import Markdown as MD


DIR = os.getcwd()
print(f'{DIR=}')

RDIR = os.path.join(DIR,'reports')
print(f'{RDIR=}')

DIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor'
RDIR='c:\\Users\\JGarza\\GitHub\\wifi_monitor\\reports'


In [360]:
def break_down_buckets(idf,column,buckets,message='',nan_value=-1):
    """
    breaks a column down into buckets/bins
    """
    idf = idf.fillna(nan_value)

    # print('',message,'\ncolumn: ',column, '\nbuckets: ', buckets)
    
    idf = pd.DataFrame(idf[column])
    idf['bucket'] = pd.cut(idf[column], bins=buckets)
    idf = idf.groupby(by='bucket').count()
    idf['percent'] = (idf[column]/idf[column].sum())*100
    idf['percent'] = idf['percent'].round(2)
    # display(idf)

    return idf

#test
# break_down_buckets(df,'AGE_YRS',[-1,0,15,25,35,45,55,65,75,85,500])

In [361]:
def bar(num,denom=100.0,length=25,fillchar='#',emptychar='_'):
    fillnum = ((int)( (num/denom) * length))
    return '[' + ( fillnum * fillchar ).ljust(length,emptychar)  + ']' # + f" {(num/denom)*100.0:.2f}%     " 

In [367]:

reports = [
    {
        "files": [
            "Error404NH_202501011257.txt",
            "Error404NH_202501020539.txt",
            "Error404NH_202501030416.txt",
            "Error404NH_202501041159.txt",
            "Error404NH_202501051039.txt",
            "Error404NH_202501060604.txt",
            "Error404NH_202501070520.txt",
            "Error404NH_202501080600.txt",
            "Error404NH_202501090551.txt",
            "Error404NH_202501100810.txt"
        ],
        "firmware":"r58881"
    },
    {
        "files": [
            "Error404NH_202501101929.txt",
            "Error404NH_202501102024.txt"
        ],
        "firmware":"r44715"
    },
    {
        "files": [
            "Error404NH_202501101939.txt",
            "Error404NH_202501102000.txt",
            "Error404NH_202501102010.txt",
            "Error404NH_202501102015.txt"
        ],
        "firmware":"r50551"
    },
]


data = {}

for r in reports:
    for file in r['files']:

        text = ''
        with open(os.path.join(RDIR, file), 'r') as f:
            text = f.read()

        try:

            wifi = re.findall('^SSID:.*\n',text)[0]
            wifi = wifi.replace('\n', '').replace('SSID: ','')
            wifi = wifi + " " + r['firmware']

            pings = re.findall('.*time=.*\n',text)
            pings = [ re.sub('.*time=','',p) for p in pings]
            pings = [ re.sub('ms.*\n','',p) for p in pings]
            pings = [ int(p) for p in pings]
            # print(*pings,sep='\n')

            timeouts = re.findall('Request timed out.\n',text)

            trycount = re.findall('(Request|Reply).*\n',text)

            if wifi in data.keys():
                pass
            else:
                data[wifi] = {}
                data[wifi]['pings'] = []
                data[wifi]['timeouts'] = []
                data[wifi]['trycount'] = []
            
            data[wifi]['pings'].extend(pings)
            data[wifi]['timeouts'].extend([len(timeouts)])
            data[wifi]['trycount'].extend([len(trycount)])   
        except Exception as e:
            print(e)
            pass

for k in data.keys():
    print('-'*20)
    print(k)
    df_pings = pd.DataFrame(data[k]['pings'])
    print(df_pings[0].describe())
    print()

    bdb = break_down_buckets(df_pings,0,[0,5,10,15,20,30,40,50,100,250,500,9999])
    bdb['bar'] = bdb.percent.apply(bar)
    for line in bdb.to_csv().split("\n"):
        print(line)
    bdb = None

    print()
    print(f'Request timed out:')
    print(f'{ sum(data[k]['timeouts']) } out of { sum(data[k]['trycount']) }')
    print(f'{ sum(data[k]['timeouts']) / sum(data[k]['trycount']) }')



--------------------
Error 404 NH r58881
count    1000.000000
mean       24.217000
std       100.082158
min         8.000000
25%        11.000000
50%        12.000000
75%        14.000000
max      2425.000000
Name: 0, dtype: float64

bucket,0,percent,bar
"(0, 5]",0,0.0,[_________________________]
"(5, 10]",215,21.5,[#####____________________]
"(10, 15]",600,60.0,[###############__________]
"(15, 20]",70,7.0,[#________________________]
"(20, 30]",44,4.4,[#________________________]
"(30, 40]",16,1.6,[_________________________]
"(40, 50]",13,1.3,[_________________________]
"(50, 100]",15,1.5,[_________________________]
"(100, 250]",17,1.7,[_________________________]
"(250, 500]",5,0.5,[_________________________]
"(500, 9999]",5,0.5,[_________________________]


Request timed out:
0 out of 1000
0.0
--------------------
Error 404 NH r44715
count     995.000000
mean       18.835176
std        44.767036
min         8.000000
25%        11.000000
50%        12.000000
75%        15.000000
max   

C:\Users\JGarza\AppData\Local\Temp\ipykernel_31528\2838929452.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()
C:\Users\JGarza\AppData\Local\Temp\ipykernel_31528\2838929452.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  idf = idf.groupby(by='bucket').count()
C:\Users\JGarza\AppData\Local\Temp\ipykernel_31528\2838929452.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  i